---
#### Corregir Elevaciones Drenajes

In [7]:
import geopandas as gpd
import rasterio
import pandas as pd
from shapely.geometry import Point
from shapely.geometry import Point, MultiLineString, LineString
import math
import numpy as np
import laspy

# --- Cargar archivos ---
shapefile_path = r"C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\Drenaje.shp"  # Ruta del shapefile de drenajes
dtm_path = r"C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\\Modelo.tif"  # Ruta del DTM
output_csv = r"C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\vertices_xyz.csv"  # Nombre del archivo CSV de salida
output_csv_2 = r"C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\vertices_OK_xyz.csv"
lasfile=r"C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\drenajes.las"
lineshp=r"C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\drenajes2.shp"
# Cargar drenajes
gdf = gpd.read_file(shapefile_path)

# Agregar un ID único para cada drenaje
gdf["ID_Drenaje"] = range(1, len(gdf) + 1)

# Extraer todos los vértices de las líneas
vertices = []
with rasterio.open(dtm_path) as dtm:
    transform = dtm.transform
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if isinstance(geom, MultiLineString):
            lines = geom.geoms
        else:
            lines = [geom]
        
        for line in lines:
            coords = np.array(line.coords)
            x, y = coords[:, 0], coords[:, 1]
            
            # Usar interpolación bilineal con sample()
            sample_points = [(xi, yi) for xi, yi in zip(x, y)]
            z_values = [val[0] for val in dtm.sample(sample_points, indexes=1)]
            
            for xi, yi, zi in zip(x, y, z_values):
                vertices.append([row["ID_Drenaje"], xi, yi, zi])

# Crear un DataFrame con los puntos
vertices_df = pd.DataFrame(vertices, columns=["ID_Drenaje", "X", "Y", "Z"])
vertices_df["abscisa"]=0
# Abscisa
initial_point = (vertices_df.iloc[0]['X'], vertices_df.iloc[0]['Y'])

# Lista para almacenar las distancias calculadas
distances = []

# Calcular la distancia entre el primer punto y los demás
for index, row in vertices_df.iterrows():
    current_point = (row['X'], row['Y'])
    # Aplicar la fórmula de distancia euclidiana
    distance = math.sqrt((current_point[0] - initial_point[0]) ** 2 + (current_point[1] - initial_point[1]) ** 2)
    # Almacenar la distancia
    distances.append(distance)
# Añadir la columna de distancias al DataFrame
vertices_df['abscisa'] = distances
# Guardar a CSV
vertices_df.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en {output_csv}")

#Corregir Elevaciones
def corregir_elevaciones(df): 
    df = df.copy().reset_index(drop=True)  # Reiniciar índice dentro del grupo
    elevaciones_corregidas = df["Z"].values.copy()

    ultimo_correcto_idx = 0
    i = 1

    while i < len(df):
        if elevaciones_corregidas[i] > elevaciones_corregidas[ultimo_correcto_idx]:
            inicio_idx = ultimo_correcto_idx

            while i < len(df) and elevaciones_corregidas[i] > elevaciones_corregidas[inicio_idx]:
                i += 1  

            if i < len(df):  
                x1, y1 = df.loc[inicio_idx, "abscisa"], elevaciones_corregidas[inicio_idx]
                x2, y2 = df.loc[i, "abscisa"], elevaciones_corregidas[i]

                for j in range(inicio_idx + 1, i):
                    x = df.loc[j, "abscisa"]
                    elevaciones_corregidas[j] = y1 + (y2 - y1) * (x - x1) / (x2 - x1)

        ultimo_correcto_idx = i  
        i += 1

    df["E1_Corregida"] = elevaciones_corregidas
    return df


# Cargar el CSV
df = pd.read_csv(output_csv)
# Aplicar la corrección por cada grupo de ID
df_corregido = df.groupby("ID_Drenaje", group_keys=False).apply(corregir_elevaciones)

# Guardar el resultado corregido
df_corregido.to_csv(output_csv_2, index=False)

# Mostrar una vista previa del resultado
print(df_corregido.head())

#Exportar a LAS

# Crear archivo LAS
header = laspy.LasHeader(point_format=3, version="1.2")  # Formato LAS 1.2
las = laspy.LasData(header)

# Asignar coordenadas desde el DataFrame
las.x = np.array(df_corregido["X"])
las.y = np.array(df_corregido["Y"])
las.z = np.array(df_corregido["E1_Corregida"])

# Guardar archivo LAS
las.write(lasfile)

print("Archivo LAS creado exitosamente.")


# Función para convertir puntos en líneas 3D
def puntos_a_linea(grupo):
    # Ordenar los puntos en el drenaje según el orden consecutivo
    grupo = grupo.sort_index()
    
    # Crear la geometría LineString con coordenadas (X, Y, Z)
    puntos = grupo.apply(lambda row: (row["X"], row["Y"], row["E1_Corregida"]), axis=1).tolist()
    return LineString(puntos)

# Crear líneas por cada ID
gdf = df_corregido.groupby("ID_Drenaje").apply(puntos_a_linea).reset_index(name="geometry")

# WKT 
crs_9377 = (
    "PROJCRS[\"MAGNA-SIRGAS 2018 / Origen-Nacional\","
    "    BASEGEOGCRS[\"MAGNA-SIRGAS 2018\","
    "        DATUM[\"Marco Geocentrico Nacional de Referencia 2018\","
    "            ELLIPSOID[\"GRS 1980\",6378137,298.257222101,"
    "                LENGTHUNIT[\"metre\",1]]],"
    "        PRIMEM[\"Greenwich\",0,"
    "            ANGLEUNIT[\"degree\",0.0174532925199433]],"
    "        ID[\"EPSG\",20046]],"
    "    CONVERSION[\"Colombia Transverse Mercator\","
    "        METHOD[\"Transverse Mercator\","
    "            ID[\"EPSG\",9807]],"
    "        PARAMETER[\"Latitude of natural origin\",4,"
    "            ANGLEUNIT[\"degree\",0.0174532925199433],"
    "            ID[\"EPSG\",8801]],"
    "        PARAMETER[\"Longitude of natural origin\",-73,"
    "            ANGLEUNIT[\"degree\",0.0174532925199433],"
    "            ID[\"EPSG\",8802]],"
    "        PARAMETER[\"Scale factor at natural origin\",0.9992,"
    "            SCALEUNIT[\"unity\",1],"
    "            ID[\"EPSG\",8805]],"
    "        PARAMETER[\"False easting\",5000000,"
    "            LENGTHUNIT[\"metre\",1],"
    "            ID[\"EPSG\",8806]],"
    "        PARAMETER[\"False northing\",2000000,"
    "            LENGTHUNIT[\"metre\",1],"
    "            ID[\"EPSG\",8807]]],"
    "    CS[Cartesian,2],"
    "        AXIS[\"northing (N)\",north,"
    "            ORDER[1],"
    "            LENGTHUNIT[\"metre\",1]],"
    "        AXIS[\"easting (E)\",east,"
    "            ORDER[2],"
    "            LENGTHUNIT[\"metre\",1]],"
    "    USAGE["
    "        SCOPE[\"Cadastre, topographic mapping.\"],"
    "        AREA[\"Colombia - onshore and offshore. Includes San Andres y Providencia, Malpelo Islands, Roncador Bank, Serrana Bank and Serranilla Bank.\"],"
    "        BBOX[-4.23,-84.77,15.51,-66.87]],"
    "    ID[\"EPSG\",9377]]"
)

# Convertir a GeoDataFrame
gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs=crs_9377)  # Usa tu EPSG

# Guardar como shapefile de líneas 3D
gdf.to_file(lineshp, driver="ESRI Shapefile")

print("Shapefile de líneas 3D creado exitosamente.")




Archivo CSV guardado en C:\Users\nicol\Downloads\Prueba_Drenajes\Prueba_Drenajes\vertices_xyz.csv
   ID_Drenaje          X          Y          Z    abscisa  E1_Corregida
0           1  4794637.0  2276937.0  811.59204   0.000000     811.59204
1           1  4794633.0  2276941.0  811.03140   5.656854     811.03140
2           1  4794631.0  2276943.0  810.37940   8.485281     810.37940
3           1  4794629.1  2276945.0  809.55304  11.243220     809.55304
4           1  4794627.4  2276947.0  808.62480  13.862179     808.62480
Archivo LAS creado exitosamente.
Shapefile de líneas 3D creado exitosamente.


## Version 2 -- Funcion Corregir elevaciones

In [ ]:
def corregir_elevaciones(df): #Funcion para encontrar errores y corregirlos.
    df = df.copy()
    elevaciones_corregidas = df["Z"].values.copy()
 
    ultimo_correcto_idx = 0  # Índice del último punto correcto
    i = 1
    
    while i < len(df):
        if elevaciones_corregidas[i] > elevaciones_corregidas[ultimo_correcto_idx]:  # Error detectado
            inicio_idx = ultimo_correcto_idx
            
            while i < len(df) and elevaciones_corregidas[i] > elevaciones_corregidas[inicio_idx]:
                i += 1  # Buscar el primer punto donde la elevación vuelve a ser decreciente

            if i < len(df):  # Se encontró un punto válido
                x1, y1 = df.loc[inicio_idx, "abscisa"], elevaciones_corregidas[inicio_idx]
                x2, y2 = df.loc[i, "abscisa"], elevaciones_corregidas[i]

                for j in range(inicio_idx + 1, i):
                    x = df.loc[j, "abscisa"]
                    elevaciones_corregidas[j] = y1 + (y2 - y1) * (x - x1) / (x2 - x1)

        ultimo_correcto_idx = i  # Actualizar el último punto correcto
        i += 1
   
    df["E1_Corregida"] = elevaciones_corregidas
    return df